In [8]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn import svm
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import cross_validate, KFold, train_test_split

In [9]:
data = pd.read_csv('train.csv')
# test_data = pd.read_csv('test.csv') # тестовый набор нельзя использовать: нет колонки SalePrice
# позже раздробим data на обучающуюся и тестовую выборки

In [10]:
yfull = data.SalePrice
Xfull = data.drop( labels = ["SalePrice"], axis = 1)


In [11]:
# data = data.dropna(axis='columns')
#Находим категориальные признаки
cat_feat = list(Xfull.dtypes[Xfull.dtypes == object].index)

#отфильтруем непрерывные признаки
num_feat = [f for f in Xfull if f not in (cat_feat + ['Id'])]

# Создаем дамми-переменные для категорий
X_dummies = pd.get_dummies(Xfull[cat_feat], columns=cat_feat)
# dummy_test = pd.get_dummies(X_test[cat_feat], columns=cat_feat)

dummy_cols = list(set(X_dummies))

X_dummies = X_dummies[dummy_cols]
# dummy_test = dummy_test[dummy_cols]

# Заменяем пропуски на специальное значение -999, чтобы деревья могли их отличить
Xfull = pd.concat([Xfull[num_feat].fillna(-999),
                     X_dummies], axis=1)


In [15]:
# Выделяем holdout-выборку 
X, Xhold, y, yhold = train_test_split(
    Xfull, yfull, random_state=42
)
X = X.reindex()
Xhold = Xhold.reindex()
y = y.reindex()
yhold = yhold.reindex()

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1095 entries, 1023 to 1126
Columns: 288 entries, MSSubClass to Exterior2nd_VinylSd
dtypes: float64(3), int64(33), uint8(252)
memory usage: 586.0 KB


In [22]:
# беру один фолд из 10 и строю на нем случайный лес, вывожу важность признаков.
# эдакая тренировка перед стекингом
# чуть поправил взятое отсюда
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold
kf = KFold(n_splits=10)
for train_index, test_index in kf.split(Xfull):
    rf = RandomForestRegressor(verbose=False, n_jobs=-1)
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = Xfull.iloc[train_index], Xfull.iloc[test_index]
    y_train, y_test = yfull[train_index], yfull[test_index]
    rf.fit(X_train,y_train)
    print("Random forest score:")
    print(rf.score(X_test,y_test))
    print("Feature importances")
    print(rf.feature_importances_)
    break # берем только один эксперимент, чтобы не заниматься никаким ансамблированием сейчас
# если нужны усредненные по k-фолдам значимости параметров, можно сделать усреднением. Но не вижу смысла.

Random forest score:
0.8584423166903885
Feature importances
[1.05217634e-03 6.38344586e-03 1.47326428e-02 5.85745989e-01
 4.34758552e-03 8.90450803e-03 5.48484583e-03 8.39705994e-03
 2.63260122e-02 3.56474846e-04 6.13197583e-03 4.14047794e-02
 2.37182101e-02 4.73209430e-02 1.03779226e-04 9.70216345e-02
 9.79696573e-04 5.83049600e-04 4.54187153e-03 4.94770298e-04
 2.16258219e-03 8.10988998e-04 6.51552552e-03 2.35319310e-03
 6.30720821e-03 2.12949139e-02 1.34653807e-02 4.52374682e-03
 3.67127812e-03 5.14938164e-04 2.75382285e-04 7.33119659e-04
 5.94037079e-06 3.50772745e-05 3.68347409e-03 1.26851995e-03
 1.20156839e-04 3.14246537e-04 4.14538149e-04 9.78342397e-06
 1.31952911e-04 6.30086164e-06 3.20077366e-05 1.90037559e-05
 4.75180479e-06 1.77680687e-07 2.64632704e-04 4.87693199e-06
 1.03221550e-04 5.37073130e-04 5.57753963e-05 2.51813641e-05
 3.71050306e-06 7.57310332e-04 1.60309097e-04 2.90046813e-07
 7.76259140e-08 2.67241510e-04 4.55174598e-04 4.58703256e-05
 2.54446462e-04 2.7459013

In [41]:
# кроссвалидация одного только случайного леса - переработаный код отсюда:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
cv_results = cross_validate(rf, Xfull, yfull, cv=10)
# sorted(cv_results.keys())
all_scores = cv_results['test_score']
from statistics import mean
# средний score по 10и фолдам для случайного леса:
mean(all_scores)
# тоже смысла не вижу - случайный лес внутри уже делит выборку 
# в итоге у каждого из деревьев совсем мало данных, 
# оттого, наверное, и score ниже чем у простого случайного леса ниже

0.8659944711082004

In [42]:
# Один случайный лес - учим на всем, кроме holdout, проверяем на holdout.
# Плюс выводим важность признаков
# Интересно, насколько скор будет отличаться у cv и такого подхода 
one_rf = RandomForestRegressor(verbose=False, n_jobs=-1)
one_rf.fit(X,y)
print(one_rf.score(Xhold,yhold))
print(one_rf.feature_importances_)
# скор получился тут су

0.8891397357874941
[2.10259687e-03 7.63572525e-03 1.74936362e-02 5.51684361e-01
 4.11054859e-03 1.16985357e-02 6.20743493e-03 3.82605725e-03
 2.95525482e-02 6.17067512e-04 4.77838697e-03 3.18086636e-02
 2.35479620e-02 2.49892103e-02 4.89140077e-05 1.21689087e-01
 6.49679585e-04 9.40679709e-05 7.52316105e-03 8.11740326e-04
 1.07274069e-03 2.99312343e-04 6.54957781e-03 4.20937210e-03
 7.49428573e-03 2.02780508e-02 1.58549054e-02 4.65439618e-03
 4.68366275e-03 7.44613784e-04 6.91944030e-05 3.17285321e-03
 9.57210023e-04 4.82861617e-05 3.96431110e-03 1.36922559e-03
 3.65464545e-04 8.83060083e-04 4.22165117e-04 6.67635259e-05
 3.38510102e-04 1.16561174e-06 2.79663244e-05 7.28317148e-06
 3.70022990e-05 4.57918512e-06 2.02974927e-04 4.91897212e-05
 4.02602522e-04 5.12651867e-04 4.93355196e-05 3.25939699e-05
 6.72251501e-06 3.54365616e-04 4.79021689e-04 2.76299620e-09
 6.35905527e-08 4.81005665e-04 8.65709074e-04 7.56013351e-05
 4.31092850e-04 1.14604509e-04 1.00761239e-05 1.87872028e-04
 7.72

In [53]:
# на трех моделях строим стекинг, 
# а потом считаем скор каждой из них в отдельности
# непонятно, как правильно совмещать кросвалидацию и holdout. 
# для меня это два независимых способа оценки качества обучения.estimators

# отсюда код взят и доработан
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html?highlight=regressor#sklearn.ensemble.StackingRegressor
from sklearn.linear_model import RidgeCV, SGDRegressor, LinearRegression
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

# X, y = load_diabetes(return_X_y=True)
estimators = [
    ('rcv', RidgeCV()),
    ('svr', LinearSVR(random_state=42)),
    ('lr', LinearRegression())
    # ('sgd', SGDRegressor(random_state=42))
]
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(random_state=42),
    cv=10,
)

reg.fit(X, y).score(Xhold, yhold)


C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinea

0.8679951319689396

In [54]:
# непонятно, как правильно совмещать кросвалидацию и holdout. 
# для меня это два независимых способа оценки качества обучения.estimators 

# кроссвалидация
for est in estimators:
    cv_results = cross_validate(est[1], Xfull, yfull, cv=10)
    print("Estimator: ")
    print(est[0])
    print("cross_val score: ")
    print(cv_results['test_score'])
    print("cross_val mean score: ")
    print(mean(cv_results['test_score']))
    # print()

Estimator: 
rcv
cross_val score: 
[0.91089043 0.90884634 0.89996691 0.78113865 0.88449541 0.88597101
 0.86924554 0.88814094 0.56787695 0.88265082]
cross_val mean score: 
0.84792230118371
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the n

In [55]:
# сравнение на holdout-выборке
for est in estimators:
    the_est = est[1]
    the_est.fit(X,y)
    print("Estimator: ")
    print(est[0])
    print(the_est.score(Xhold,yhold))
    # print(one_rf.feature_importances_)  
    # print("cross_val score: ")
    # print(cv_results['test_score'])
    # print("cross_val mean score: ")
    # print(mean(cv_results['test_score']))
    # # print()

Estimator: 
rcv
0.8807909113996806
Estimator: 
svr
0.8030962274499284
Estimator: 
lr
0.4874106879591913
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


В итоге RidgeCV при сравнении на holdout дал бОльший score, чем стекинг.
Значит стекинг построен неверно. Как его исправить.